#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [5]:
import requests
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [6]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [7]:
def resolve_embeddings():
    try:
        response=requests.get("http://127.0.0.1:11434/api/tags",timeout=3)
        response.raise_for_status()
        return OllamaEmbeddings()
    except Exception:
        try:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        except Exception as embedding_error:
            raise RuntimeError("Neither Ollama nor HuggingFace embeddings are available. Start Ollama via 'ollama serve' or install 'sentence-transformers'.") from embedding_error

embeddings=resolve_embeddings()
db=FAISS.from_documents(docs,embeddings)
db

C:\Users\shankgo\AppData\Local\Temp\ipykernel_15696\2690073802.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [9]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [10]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [11]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='6bbb6f66-1da5-4f83-bce3-ad061ec25cff', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(1.3070925)),
 (Document(id='7f5cbe5b-f201-4050-953d-1947d83befa6', metadata={'source': 's

In [12]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.049689557403326035,
 0.10655158013105392,
 0.0015430999919772148,
 -0.013194359838962555,
 -0.0443878173828125,
 0.05857320502400398,
 0.046664610505104065,
 -0.024530639871954918,
 0.004829468205571175,
 0.0203399620950222,
 -0.03410930931568146,
 0.038737911731004715,
 0.016618911176919937,
 -0.018834847956895828,
 -0.007143062073737383,
 0.006980108097195625,
 0.020850053057074547,
 -0.03520536050200462,
 0.004866405390202999,
 -0.0072698211297392845,
 0.07890864461660385,
 0.05607706680893898,
 0.06984829902648926,
 0.01655665412545204,
 0.013394546695053577,
 -0.012157349847257137,
 -0.020828310400247574,
 0.05277486890554428,
 -0.020383469760417938,
 0.02614760771393776,
 0.045908521860837936,
 -0.017534716054797173,
 0.016683531925082207,
 0.002140582539141178,
 0.006522985175251961,
 0.0013917393516749144,
 -0.05417174473404884,
 -0.016581464558839798,
 -0.011631478555500507,
 -0.020481346175074577,
 -0.0529186874628067,
 -0.03948397561907768,
 0.01654801517724991,
 0.04053

In [13]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='6bbb6f66-1da5-4f83-bce3-ad061ec25cff', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='7f5cbe5b-f201-4050-953d-1947d83befa6', metadata={'source': 'speech.txt'}, page_content='T

In [14]:
### Saving And Loading
db.save_local("faiss_index")

In [15]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [16]:
docs

[Document(id='6bbb6f66-1da5-4f83-bce3-ad061ec25cff', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='7f5cbe5b-f201-4050-953d-1947d83befa6', metadata={'source': 'speech.txt'}, page_content='T